In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

In [114]:
class ExoplanetDataset(Dataset):
    """Exoplanet dataset."""

    def __init__(self, X, y, standardize=True, totensor=True):
        """
        Args:
            csv_file (string): Path to the csv file.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.X = X
        self.y = y
        self.standardize = standardize
        self.totensor = totensor

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        x = self.X[idx]

        if self.standardize:
            x = (x - np.mean(x)) / np.std(x)

        if self.totensor:
            x = torch.tensor(x, dtype=torch.float).unsqueeze(0)

        sample = (int(self.y[idx]), x)
        return sample

In [121]:
data = np.genfromtxt("kepler-labelled-time-series-data/exoTrain.csv", delimiter=',')
data = data[~np.isnan(data).any(axis=1)]
X = data[:, 1:]
y = data[:, 0] - 1

dataset = ExoplanetDataset(X, y)
cuda_available = torch.cuda.is_available()

device = torch.device("cuda" if cuda_available else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda_available else {}
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, **kwargs)

In [122]:
def train(model, device, train_loader, criterion, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (target, data) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [123]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 20, kernel_size=5, stride=1)
        self.maxpool1 = nn.MaxPool1d(kernel_size=5)
        self.conv2 = nn.Conv1d(20, 50, kernel_size=5, stride=1)
        self.maxpool2 = nn.MaxPool1d(kernel_size=3)
        self.conv3 = nn.Conv1d(50, 100, kernel_size=3, stride=1)
        self.maxpool3 = nn.MaxPool1d(kernel_size=3)
        
        self.fc1 = nn.Linear(100 * 210, 500)
        self.fc2 = nn.Linear(500, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = F.relu(self.conv2(x))
        self.maxpool2(x)
        x = F.relu(self.conv3(x))
        x = self.maxpool3(x)
        
        x = x.view(-1, 100 * 210)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [124]:
model = Net().to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [125]:
for idx, (target, data) in enumerate(train_loader):
    print(model(data).shape)
    break

torch.Size([32, 2])


In [126]:
for epoch in range(10):
    train(model, device, train_loader, criterion, optimizer, epoch, 100)

Train Epoch: 0 [0/5087 (0%)]	Loss: 0.720457
Train Epoch: 0 [3200/5087 (63%)]	Loss: 0.006997
Train Epoch: 1 [0/5087 (0%)]	Loss: 0.001914
Train Epoch: 1 [3200/5087 (63%)]	Loss: 0.001927
Train Epoch: 2 [0/5087 (0%)]	Loss: 0.000657
Train Epoch: 2 [3200/5087 (63%)]	Loss: 0.008815
Train Epoch: 3 [0/5087 (0%)]	Loss: 0.105413
Train Epoch: 3 [3200/5087 (63%)]	Loss: 0.005348
Train Epoch: 4 [0/5087 (0%)]	Loss: 0.149000
Train Epoch: 4 [3200/5087 (63%)]	Loss: 0.244002
Train Epoch: 5 [0/5087 (0%)]	Loss: 0.002694
Train Epoch: 5 [3200/5087 (63%)]	Loss: 0.005491
Train Epoch: 6 [0/5087 (0%)]	Loss: 0.001213


KeyboardInterrupt: 

tensor([[[-3.0445, -2.5404, -2.5358,  ..., -2.7763, -2.9808, -2.8239]],

        [[ 0.1706,  0.4782, -0.2836,  ..., -0.3786,  2.3318,  0.0289]],

        [[ 0.5657, -0.2222,  0.1859,  ...,  1.1242,  0.3966,  0.2730]],

        ...,

        [[-0.7643, -1.2100, -1.2651,  ...,  0.5843, -0.6019, -0.5981]],

        [[ 7.5198,  7.4820,  6.9032,  ..., -0.2731, -0.3784,  0.0928]],

        [[-0.3860,  0.1872, -0.1681,  ..., -0.8127, -0.7653, -0.7163]]])